Now this file will all the data cleaning done on the dataset according to the findings of the EDA.


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_raw = pd.read_csv("electricity_consumption_optimization_dataset.csv")

df = df_raw.copy()
print('Shape:- ', df.shape)
df.info()


In [ ]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

df["hour"] = df["Timestamp"].dt.hour
df["day"] = df["Timestamp"].dt.day
df["month"] = df["Timestamp"].dt.month
df = df.drop(columns='Timestamp')
df



Removing negative energy values ( Since energy can't be in -ve)

first i tried to do this but then it removed almost 7k+ rows 
so now i am clipping them instead of removing

In [ ]:
energy_cols = [
    "Energy Consumption (kWh)",
    "Lighting Consumption (kWh)",
    "HVAC Consumption (kWh)",
    "Local Energy Production (kWh)",
    "Room-Level Energy Consumption (kWh)",
    "Zonal Heating/Cooling Data (kWh)",
    "Smart Plug Usage (kWh)",
    "Historical Energy Consumption (kWh)"
]

for col in energy_cols:
    df[col] = df[col].clip(lower=0)
print('Shape', df.shape)

Fixing occupancy rate ( setting the rabge of it between 0 to 100)

In [ ]:
df = df[
    (df["Occupancy Rate (%)"] >= 0) &
    (df["Occupancy Rate (%)"] <= 100)
]
print(df['Occupancy Rate (%)'])

In [ ]:
percentage_cols = [
    "Window-to-Wall Ratio (%)",
    "Energy Savings Target (%)",
    "Energy Savings Potential (%)"
]

for col in percentage_cols:
    df= df[(df[col] >= 0) & (df[col] <= 100)]

x = df[[
    "Window-to-Wall Ratio (%)",
    "Energy Savings Target (%)",
    "Energy Savings Potential (%)"
]]
print(x)



In [ ]:
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns

def remove_outliers_iqr(data, columns):
    for col in columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower) & (data[col] <= upper)]
    return data

df = remove_outliers_iqr(df, numeric_cols)
print("Shape after outlier removal:", df.shape)

In [ ]:
binary_cols = [
    "Power Outage Indicator",
    "Maintenance Status",
    "Demand Response Participation",
    "Electric Vehicle Charging Status",
    "Peak Demand Reduction Indicator"
]

df[binary_cols] = df[binary_cols].astype(int)
y =df[ [
    "Power Outage Indicator",
    "Maintenance Status",
    "Demand Response Participation",
    "Electric Vehicle Charging Status",
    "Peak Demand Reduction Indicator"
]]
print(y)

In [ ]:
print(df.columns.tolist())



In [ ]:
categorical_cols = [
    "Building Type",
    "Occupancy Schedule",
    "Building Orientation",
    "Carbon Emission Reduction Category"
]

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)



Now doing feature scaling by using standard scaler 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_features = scaler.fit_transform(df)

df_scaled = pd.DataFrame(scaled_features, columns=df.columns)

print("Final cleaned shape:", df_scaled.shape)

In [ ]:
df.describe().T

Checking Multicollinearity

In [ ]:
import numpy as np

corr_matrix = df.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

high_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
print("Highly correlated columns:", high_corr)